In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
url = '/content/drive/MyDrive/datasets/diabetes.csv'

import pandas as pd
df = pd.read_csv(url)
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [9]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [10]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


## check outlier if there is

In [11]:
df.Outcome.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [12]:
268/500

0.536

In [13]:
X = df.drop('Outcome', axis=1)
X[0:5]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [14]:
y = df.Outcome
y[0:5]

0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled[0:5]

array([[ 0.63994726,  0.84832379,  0.14964075,  0.90726993, -0.69289057,
         0.20401277,  0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575,  0.53090156, -0.69289057,
        -0.68442195, -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, -1.28821221, -0.69289057,
        -1.10325546,  0.60439732, -0.10558415],
       [-0.84488505, -0.99820778, -0.16054575,  0.15453319,  0.12330164,
        -0.49404308, -0.92076261, -1.04154944],
       [-1.14185152,  0.5040552 , -1.50468724,  0.90726993,  0.76583594,
         1.4097456 ,  5.4849091 , -0.0204964 ]])

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, random_state=10) # stratify if there is imbalance in outcome of unique values of traget

In [17]:
X_train.shape

(576, 8)

In [18]:
X_test.shape

(192, 8)

In [21]:
y_train.value_counts()

0    375
1    201
Name: Outcome, dtype: int64

In [22]:
201 / 375

0.536

In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

score = cross_val_score(DecisionTreeClassifier(), X, y, cv=5)
scoreScaled = cross_val_score(DecisionTreeClassifier(), X_scaled, y, cv=5)

print(f"Without Scale: {score.mean()}")
print(f"With Scale: {scoreScaled.mean()}")



Without Scale: 0.7045666751549104
With Scale: 0.7227569815805109


In [30]:
from sklearn.ensemble import BaggingClassifier

bag_model = BaggingClassifier(
    base_estimator = DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=0
)
bag_model.fit(X_train, y_train)
bag_model.oob_score_

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


0.7534722222222222

In [31]:
bag_model.score(X_test, y_test)

0.7760416666666666

In [32]:
bag_model = BaggingClassifier(
    base_estimator = DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=0
)

scores = cross_val_score(bag_model, X, y, cv=5)
scores.mean()

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


0.7578728461081402

In [33]:
# high variance then use bagging

In [34]:
from sklearn.ensemble import RandomForestClassifier

scores = cross_val_score(RandomForestClassifier(), X, y, cv=5)
scores.mean()

0.7604872251931075

In [88]:
url = '/content/drive/MyDrive/datasets/heart.csv'

import pandas as pd
df = pd.read_csv(url)
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [89]:
df.shape

(918, 12)

In [90]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [91]:
Z_threshold = 3

cols_Ztest = ['RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak']

for cols in cols_Ztest:
  std_col = df[cols].std()
  df_temp = df[df[cols] > (df[cols].mean() + Z_threshold*std_col)]

  if df_temp.empty:
    print(f"No outliers for {cols}")

  else:
    print(f"Outliers for {cols} done")
    df = df[df[cols] <= (df[cols].mean() + Z_threshold*std_col)]

Outliers for RestingBP done
Outliers for Cholesterol done
No outliers for FastingBS
No outliers for MaxHR
Outliers for Oldpeak done


In [92]:
df.shape

(902, 12)

In [93]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [94]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

cols_to_encode = ['ExerciseAngina', 'ST_Slope', 'RestingECG', 'Sex', 'ChestPainType']

for col in cols_to_encode:
    df[col] = label_encoder.fit_transform(df[col])

df.head()


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,1,140,289,0,1,172,0,0.0,2,0
1,49,0,2,160,180,0,1,156,0,1.0,1,1
2,37,1,1,130,283,0,2,98,0,0.0,2,0
3,48,0,0,138,214,0,1,108,1,1.5,1,1
4,54,1,2,150,195,0,1,122,0,0.0,2,0


In [95]:
X = df.drop('HeartDisease', axis=1)
y = df.HeartDisease

In [96]:
print(X[:5])
print(y[:5])

   Age  Sex  ChestPainType  RestingBP  Cholesterol  FastingBS  RestingECG  \
0   40    1              1        140          289          0           1   
1   49    0              2        160          180          0           1   
2   37    1              1        130          283          0           2   
3   48    0              0        138          214          0           1   
4   54    1              2        150          195          0           1   

   MaxHR  ExerciseAngina  Oldpeak  ST_Slope  
0    172               0      0.0         2  
1    156               0      1.0         1  
2     98               0      0.0         2  
3    108               1      1.5         1  
4    122               0      0.0         2  
0    0
1    1
2    0
3    1
4    0
Name: HeartDisease, dtype: int64


In [102]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [103]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_scaled

array([[-1.73318169, -1.88005085,  1.28768629, ..., -0.83454813,
        -0.84905528,  1.07966914],
       [-0.36494182,  0.53190051, -0.82716978, ...,  1.19825323,
        -0.35337199, -0.60526906],
       [ 0.79279961,  0.53190051, -0.82716978, ...,  1.19825323,
        -0.84905528,  1.07966914],
       ...,
       [ 0.47705195, -1.88005085,  1.28768629, ..., -0.83454813,
        -0.84905528,  1.07966914],
       [-0.2596926 ,  0.53190051, -0.82716978, ...,  1.19825323,
         0.3405846 , -0.60526906],
       [-1.41743403,  0.53190051, -0.82716978, ...,  1.19825323,
         0.14231129, -0.60526906]])

In [104]:
from sklearn.svm import SVC

model_svm = SVC()
model_svm.fit(X_scaled, y_train)
model_svm.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


0.5303867403314917

In [109]:
from sklearn.ensemble import BaggingClassifier

bag_model = BaggingClassifier(
    estimator=SVC(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=42
)

bag_modelScaled = BaggingClassifier(
    estimator=SVC(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=42
)

bag_model.fit(X_train, y_train)
print(f"oob score not scaled: {bag_model.oob_score_}")

bag_modelScaled.fit(X_scaled, y_train)
print(f"oob score scaled: {bag_modelScaled.oob_score_}")

print(f"not scaled accuracy: {bag_model.score(X_test, y_test)}")
print(f"scaled accuracy: {bag_modelScaled.score(X_test_scaled, y_test)}")


oob score not scaled: 0.7142857142857143
oob score scaled: 0.8682385575589459
not scaled accuracy: 0.7403314917127072
scaled accuracy: 0.8784530386740331


In [110]:
from sklearn.ensemble import BaggingClassifier

bag_model = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=42
)

bag_modelScaled = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=42
)

bag_model.fit(X_train, y_train)
print(f"oob score not scaled: {bag_model.oob_score_}")

bag_modelScaled.fit(X_scaled, y_train)
print(f"oob score scaled: {bag_modelScaled.oob_score_}")

print(f"not scaled accuracy: {bag_model.score(X_test, y_test)}")
print(f"scaled accuracy: {bag_modelScaled.score(X_test_scaled, y_test)}")


oob score not scaled: 0.8432732316227461
oob score scaled: 0.8432732316227461
not scaled accuracy: 0.8784530386740331
scaled accuracy: 0.8729281767955801


In [108]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming you have X and y already defined

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features (X) using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create the BaggingClassifier with the scaled features
bag_modelScaled = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=42
)

# Fit the BaggingClassifier on the scaled training data
bag_modelScaled.fit(X_train_scaled, y_train)

# Evaluate the BaggingClassifier on the scaled test data
scaled_accuracy = bag_modelScaled.score(X_test_scaled, y_test)

print(f"Scaled accuracy: {scaled_accuracy}")


Scaled accuracy: 0.8729281767955801


# DONT FORGET X_test_scaled = scaler.transform(X_test)
WHEN SCALING


# BAGGING HELPS AND STANDARDSCALER DOES TOO BUT DONT FORGET THE MSG ABOVE